In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import env
import acquire
from scipy import stats
import sklearn.preprocessing
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV
from numpy import percentile


from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge, SGDRegressor, LinearRegression, Lasso, Ridge, ElasticNet, RidgeCV
from sklearn.svm import SVR, LinearSVR, NuSVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures, RobustScaler, QuantileTransformer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, make_scorer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.neural_network import MLPRegressor
import math
import featuretools as ft
import tabulate
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Wrangling
import pandas as pd
import numpy as np

# Exploring
import scipy.stats as stats

# Visualizing
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# default pandas decimal number display format
pd.options.display.float_format = '{:20,.2f}'.format
import datatile
from datatile.summary.df import DataFrameSummary


sns.set()

In [14]:
df = acquire.get_zillow_all()

Getting a fresh copy from SQL database...
Saving to csv...


In [15]:
df.head()

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,1727539,14297519,NaN,NaN,NaN,3.50,4.00,NaN,NaN,3.50,...,"60,590,630,072,012.00",0.03,2017-01-01,None,None,None,None,Single Family Residential,None,None
1,1387261,17052889,NaN,NaN,NaN,1.00,2.00,NaN,NaN,1.00,...,"61,110,010,023,006.00",0.06,2017-01-01,None,None,None,None,Single Family Residential,None,None
2,11677,14186244,NaN,NaN,NaN,2.00,3.00,NaN,NaN,2.00,...,"60,590,218,022,012.00",0.01,2017-01-01,None,None,None,None,Single Family Residential,None,None
3,2288172,12177905,NaN,NaN,NaN,3.00,4.00,NaN,8.00,3.00,...,"60,373,001,001,006.00",-0.10,2017-01-01,None,None,None,Central,Single Family Residential,None,None
4,1970746,10887214,1.00,NaN,NaN,3.00,3.00,NaN,8.00,3.00,...,"60,371,236,012,000.00",0.01,2017-01-01,Central,None,None,Central,Condominium,None,None


In [16]:
df.duplicated().sum()

0

In [17]:
dfs = DataFrameSummary(df, plot=True)

In [18]:
dfs.columns_stats

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
counts,77380,77380,24953,206,50,77380,77380,15,49671,76771,...,77136,77380,77380,24953,206,15,49439,77380,50,222
uniques,77380,77380,5,5,43,22,16,2,12,21,...,39006,76836,264,5,5,2,10,13,1,4
missing,0,0,52427,77174,77330,0,0,77365,27709,609,...,244,0,0,52427,77174,77365,27941,0,77330,77158
missing_perc,0%,0%,67.75%,99.73%,99.94%,0%,0%,99.98%,35.81%,0.79%,...,0.32%,0%,0%,67.75%,99.73%,99.98%,36.11%,0%,99.94%,99.71%
types,numeric,numeric,numeric,numeric,numeric,numeric,numeric,bool,numeric,numeric,...,numeric,numeric,categorical,categorical,categorical,bool,categorical,categorical,constant,categorical


In [19]:
df.isnull().sum()

id                              0
parcelid                        0
airconditioningtypeid       52427
architecturalstyletypeid    77174
basementsqft                77330
                            ...  
buildingclassdesc           77365
heatingorsystemdesc         27941
propertylandusedesc             0
storydesc                   77330
typeconstructiondesc        77158
Length: 68, dtype: int64

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77380 entries, 0 to 77379
Data columns (total 68 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            77380 non-null  int64  
 1   parcelid                      77380 non-null  int64  
 2   airconditioningtypeid         24953 non-null  float64
 3   architecturalstyletypeid      206 non-null    float64
 4   basementsqft                  50 non-null     float64
 5   bathroomcnt                   77380 non-null  float64
 6   bedroomcnt                    77380 non-null  float64
 7   buildingclasstypeid           15 non-null     float64
 8   buildingqualitytypeid         49671 non-null  float64
 9   calculatedbathnbr             76771 non-null  float64
 10  decktypeid                    614 non-null    float64
 11  finishedfloor1squarefeet      6023 non-null   float64
 12  calculatedfinishedsquarefeet  77184 non-null  float64
 13  f

3. Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values. 

In [21]:
def row_nan_pct(df):
    # creates a dataframe for storing nan in columns and percent of rows with nan
    df_miss = pd.DataFrame()
    # creates columns for total missing values per column
    df_miss['num_rows_missing'] = df.isnull().sum()
    # creates columns for percent of total missing values per column
    df_miss['pct_rows_missing'] = df.isnull().mean() * 100
    # returns a dataframe with columns for total missing values per column and percent of total missing values per column
    return df_miss.sort_values(by='pct_rows_missing', ascending=False)

In [22]:
row_nan_pct(df)

,num_rows_missing,pct_rows_missing
buildingclassdesc,77365,99.98
buildingclasstypeid,77365,99.98
finishedsquarefeet13,77339,99.95
storytypeid,77330,99.94
basementsqft,77330,99.94
...,...,...
latitude,0,0.00
fips,0,0.00
bedroomcnt,0,0.00
bathroomcnt,0,0.00


In [23]:
def col_nan_pct(df):
    # creates a dataframe for storing nan in columns and percent of rows with nan
    df_miss_col = pd.DataFrame()
    # creates columns for total missing values per column
    df_miss_col['num_cols_missing'] = df.apply(lambda x: x.isnull().sum(axis=0), axis=1)
    # creates columns for percent of total missing values per column
    df_miss_col['pct_cols_missing'] = df.isnull().mean(axis=1) * 100

    df_miss_col['num_rows'] = df.apply(lambda row: row.isnull().sum(axis=1), axis=0)
    # returns a dataframe with columns for total missing values per column and percent of total missing values per column
    return df_miss_col.sort_values(by='num_cols_missing', ascending=True)

In [ ]:
col_nan_pct(df)

In [26]:
df.propertylandusetypeid.drop(row=! 261, axis=0, inplace=True)

TypeError: filter expected 2 arguments, got 1